In [1]:
#Reading the Normal - csv files from the directory

import os
import glob
import pandas as pd

def concatenate(indir='/Users/vamsi_5/Downloads/normalcsvs'):
    os.chdir(indir)
    fileList = glob.glob("*.csv")
    List=[]
    for i in fileList:
        print(i)
        df = pd.read_csv(i)
        List.append(df)
    concatdF = pd.concat(List,axis=0)
    
    return concatdF
    
    


In [2]:
#Reading the Malware - csv files from the directory

def concatenate2(indir='/Users/vamsi_5/Downloads/malwareCSV'):
    os.chdir(indir)
    fileList = glob.glob("*.csv")
    List=[]
    for i in fileList:
        print(i)
        df = pd.read_csv(i)
        List.append(df)
    concatdF2 = pd.concat(List,axis=0)
    
    return concatdF2
    

In [3]:
#Fucntion Call
Normal_DF = concatenate()
Malware_DF = concatenate2()

updated_2017-05-01_normal-2.pcap.csv
updated_2017-04-25_win-normal.pcap.csv
updated_2017_04_30-normal.pcap.csv
updated_2017-04-28_normal.pcap.csv
updated_2017-04-18_win-normal.pcap.csv
updated_2017-04-19_win-normal.pcap.csv
malwareCapture4.csv
malwareCapture5.csv
malwareCapture7.csv
malwareCapture6.csv
malwareCapture2.csv
malwareCapture3.csv
malwareCapture1.csv
malwareCapture11.csv
malwareCapture12.csv
malwareCapture13.csv
malwareCapture8.csv


In [5]:
#Reading from multiple files, index re-starts at every new file. To have every thing in sequence
Normal_DF.reset_index(drop=True, inplace=True)
Malware_DF.reset_index(drop=True, inplace=True)

In [6]:
Normal_DF.columns

Index(['IP Source', 'IP Destination', 'Source Port', 'Destination Port',
       'TCP Flag Nonce', 'TCP Flag Congestion Window Reduced',
       'TCP Flag ECN-Echo', 'TCP Flag Urgent', 'TCP Flag Acknowledgement',
       'TCP Flag Push', 'TCP Flag Reset', 'TCP Flag Syn', 'TCP Flag Fin',
       ' Malicious'],
      dtype='object')

In [7]:
#No of unique values are too much, Can we really label encode IP addresses ?
print(Normal_DF['IP Source'].nunique())
print(Malware_DF['IP Source'].nunique())

3881
1016


In [8]:
#Splitting the IP source into 4 columns based on '.'
Normal_DF['IP1'],Normal_DF['IP2'],Normal_DF['IP3'],Normal_DF['IP4'] = Normal_DF['IP Source'].str.split('.', 0).str
Normal_DF['IP5'],Normal_DF['IP6'],Normal_DF['IP7'],Normal_DF['IP8'] = Normal_DF['IP Destination'].str.split('.', 0).str

In [9]:
#Splitting the IP source into 4 columns based on '.'
Malware_DF['IP1'],Malware_DF['IP2'],Malware_DF['IP3'],Malware_DF['IP4'] = Malware_DF['IP Source'].str.split('.', 0).str
Malware_DF['IP5'],Malware_DF['IP6'],Malware_DF['IP7'],Malware_DF['IP8'] = Malware_DF['IP Destination'].str.split('.', 0).str

In [10]:
#Converting the columns to numeric.
cols = ['IP1','IP2','IP3','IP4','IP5','IP6','IP7','IP8']
for i in cols:
    Normal_DF[i] = pd.to_numeric(Normal_DF[i])
    Malware_DF[i] = pd.to_numeric(Malware_DF[i])
    

In [11]:
#Converting the columns to boolean
cols2 = ['TCP Flag Nonce', 'TCP Flag Congestion Window Reduced',
       'TCP Flag ECN-Echo', 'TCP Flag Urgent', 'TCP Flag Acknowledgement',
       'TCP Flag Push', 'TCP Flag Reset', 'TCP Flag Syn', 'TCP Flag Fin',
       ' Malicious']

for i in cols2:
    Normal_DF[i] = Normal_DF[i].astype('bool')
    Malware_DF[i] = Malware_DF[i].astype('bool')

In [12]:
#Dropping the columns
Normal_DF.drop(['IP Source', 'IP Destination'], axis=1, inplace=True)
Malware_DF.drop(['IP Source', 'IP Destination'], axis=1,inplace=True)

In [13]:
#Concatenating the Normal and Malware DataFrames
Final_DF = pd.concat([Normal_DF,Malware_DF])

In [14]:
#Resetting the index
Final_DF.reset_index(drop=True, inplace=True)
Final_DF.head()

,Source Port,Destination Port,TCP Flag Nonce,TCP Flag Congestion Window Reduced,TCP Flag ECN-Echo,TCP Flag Urgent,TCP Flag Acknowledgement,TCP Flag Push,TCP Flag Reset,TCP Flag Syn,TCP Flag Fin,Malicious,IP1,IP2,IP3,IP4,IP5,IP6,IP7,IP8
0,49157,80,False,False,False,False,False,False,False,True,False,False,10,0,2,15,161,69,12,13
1,80,49157,False,False,False,False,True,False,False,True,False,False,161,69,12,13,10,0,2,15
2,49157,80,False,False,False,False,True,False,False,False,False,False,10,0,2,15,161,69,12,13
3,49157,80,False,False,False,False,True,False,False,False,True,False,10,0,2,15,161,69,12,13
4,80,49157,False,False,False,False,True,False,False,False,False,False,161,69,12,13,10,0,2,15


In [16]:
#Importing the Machine learning Algorithms from the sklearn library
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [17]:
X = Final_DF[['Source Port', 'Destination Port', 'TCP Flag Nonce',
       'TCP Flag Congestion Window Reduced', 'TCP Flag ECN-Echo',
       'TCP Flag Urgent', 'TCP Flag Acknowledgement', 'TCP Flag Push',
       'TCP Flag Reset', 'TCP Flag Syn', 'TCP Flag Fin','IP1',
       'IP2', 'IP3', 'IP4', 'IP5', 'IP6', 'IP7', 'IP8']]
y = Final_DF[' Malicious']

In [18]:
#Kfold cross validation Technique

from sklearn import model_selection

kfold = model_selection.KFold(n_splits=10, random_state=42)
model = LogisticRegression()
model2 = RandomForestClassifier()

In [20]:
#Results for Logistic Regression
results = model_selection.cross_val_score(model, X, y, cv=kfold)
print("Accuracy: %.3f%% (%.3f%%)" % (results.mean()*100.0, results.std()*100.0))

Accuracy: 99.410% (0.708%)


In [21]:
#Results for Random Forest Classifier
results2 = model_selection.cross_val_score(model2, X, y, cv=kfold)
print("Accuracy: %.3f%% (%.3f%%)" % (results2.mean()*100.0, results2.std()*100.0))

Accuracy: 100.000% (0.000%)


In [22]:
#Hold Out Validation - Test and train Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


In [23]:
#Logistic Regression model
logmodel = LogisticRegression()
logmodel.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [24]:
#Predicting the test data
predictions = logmodel.predict(X_test)

In [25]:
#Classification Report 
from sklearn.metrics import classification_report
print(classification_report(y_test,predictions))


             precision    recall  f1-score   support

      False       0.99      1.00      1.00    889531
       True       1.00      0.99      0.99    559724

avg / total       0.99      0.99      0.99   1449255



In [38]:
#Confusion matrix 
from sklearn.metrics import confusion_matrix
CM = confusion_matrix(y_test,predictions)
Accuracy = (CM[0][0] + CM[1][1])/CM.sum()
print(CM)

[[887822   1709]
 [  5761 553963]]
